In [1]:
DATA_NAME = 'spaceNet-full' 
TRANSFORM = 'wavelet-horizVert'
CHANNEL = 'blue'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_blue_horizVert_wavelet_full_spaceNet.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,136799.90,0.64,0.85,-0.18,0.15,0.06,-0.01,-0.00
3,0.64,31080.31,-0.64,-0.07,0.04,-0.01,0.00,-0.00
4,0.85,-0.64,8250.20,-0.04,0.02,0.04,0.00,0.00
5,-0.18,-0.07,-0.04,1916.65,-0.02,-0.02,-0.00,-0.00
6,0.15,0.04,0.02,-0.02,350.04,0.00,0.00,0.00
7,0.06,-0.01,0.04,-0.02,0.00,55.12,-0.00,0.00
8,-0.01,0.00,0.00,-0.00,0.00,-0.00,7.67,0.00
9,-0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,0.72


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,0.00001,0.00003,-0.00001,0.00002,0.00002,-0.00001,-0.00001
3,0.00001,1.00000,-0.00004,-0.00001,0.00001,-0.00000,0.00001,-0.00002
4,0.00003,-0.00004,1.00000,-0.00001,0.00001,0.00006,0.00000,0.00000
5,-0.00001,-0.00001,-0.00001,1.00000,-0.00002,-0.00005,-0.00000,-0.00000
6,0.00002,0.00001,0.00001,-0.00002,1.00000,0.00002,0.00001,0.00001
7,0.00002,-0.00000,0.00006,-0.00005,0.00002,1.00000,-0.00002,0.00006
8,-0.00001,0.00001,0.00000,-0.00000,0.00001,-0.00002,1.00000,0.00002
9,-0.00001,-0.00002,0.00000,-0.00000,0.00001,0.00006,0.00002,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

1.7980115197259967

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[2.34204377e+09 1.18418760e+08 8.10463842e+06 4.19756891e+05
 1.31362650e+04 2.89965860e+02 4.21848951e+00 1.69775467e-33]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999379,-0.034134,-0.008557,-0.001983,-0.000361,-0.000057,-0.000008,-7.584793e-07
1,0.033665,0.998237,-0.047761,-0.010226,-0.001858,-0.000293,-0.000041,-3.942287e-06
2,0.010041,0.046868,0.997586,-0.049508,-0.008471,-0.001326,-0.000185,-1.744414e-05
3,0.002799,0.012354,0.048485,0.997583,-0.047586,-0.007195,-0.000999,-9.421724e-05
4,0.000636,0.002802,0.010572,0.046629,0.997365,-0.053996,-0.007269,-6.844094e-04
5,0.000133,0.000587,0.002204,0.009522,0.052931,0.996007,-0.070924,-6.480045e-03
6,0.000028,0.000122,0.000459,0.001980,0.010837,0.069502,0.993039,-9.444828e-02
7,0.000005,0.000023,0.000087,0.000375,0.002054,0.013039,0.093747,9.955084e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0006213974644870834,
 0.0017628546079837548,
 0.002413925411264062,
 0.0024171622520079428,
 0.0026350294562731413,
 0.0039928180175374806,
 0.006960512534560759,
 0.0044915600064485695]